In [1]:
# Pre-Process CMV Data
# ====================
import pandas as pd
import json
import re
import os
from IPython.display import Markdown
import random

In [2]:
print(os.getcwd())

/Users/joshua.sheppard/PycharmProjects/countaBot/src/EDA


In [3]:
conan = [json.loads(ln) for ln in open("../data/raw/CONAN.json", "r")]
conan_multi = [json.loads(ln) for ln in open("../data/raw/multitarget_CONAN.json", "r")]

print(len(conan[0]["conan"]))
print(len(conan_multi[0]))

conan
conan_multi

14988
5003


[{'0': {'HATE_SPEECH': "Maybe the UN could talk to those asian and african nations responsible for 90%+ of the pollution in the oceans' instead of insisting on this bullshit about climate change.",
   'COUNTER_NARRATIVE': "You're right, ocean pollution is one of the main environmental problems we face, however, climate change is a global problem, and everyone should help to combat it, no matter where in the world you are from!",
   'TARGET': 'other',
   'VERSION': 'V1'},
  '1': {'HATE_SPEECH': '‘Juice (Jews) are worse that nukes.’ Caption to a meme showing Hiroshima 1945 and present day, contrast against pictures of Detroit in 1945 and present day.’',
   'COUNTER_NARRATIVE': "You'd rather be nuked then?",
   'TARGET': 'JEWS',
   'VERSION': 'V1'},
  '2': {'HATE_SPEECH': 'Foreigners on UK benefits leaps 41% in 5 years with over 400K now handed payouts. This nonsense has got to stop.',
   'COUNTER_NARRATIVE': 'You seem to be worried about government expenditure… it has actually been shown

In [4]:
ids = []
hate = []
counter = []
type = []
sub_type = []

for _ in conan[0]["conan"]:
    if _["cn_id"][0:3] == "ENT":
        ids.append(_["cn_id"])
        hate.append(_["hateSpeech"])
        counter.append(_["counterSpeech"])
        # type.append(_["hsType"])
        # sub_type.append(_["hsSubType"])

print(len(hate))

3864


In [5]:
conan[0]

{'conan': [{'cn_id': 'ENT1ST0001HS0033CN000021',
   'hateSpeech': 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
   'counterSpeech': 'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.',
   'hsType': 'Islamophobia',
   'hsSubType': 'crimes',
   'cnType': 'facts',
   'age': 61.0,
   'gender': 'male',
   'educationLevel': 'Bachelor'},
  {'cn_id': 'ENT1ST0001HS0033CN000021P1',
   'hateSpeech': "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
   'counterSpeech': 'To be fair, the OFSTED report is more concerned with lack of enforcement and less about focussing on the practice of any particular faith.',
   'hsType': 'Islamophobia',
   'hsSubType': 'crimes',
   'cnType': 'facts',
   'age': 61.0,
   'gender': 'male',
   'educationLevel': 'Bachelor'},
 

In [6]:
sub_type
counter
type
hate

['According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
 "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
 'A school in Birningham is still separating girls from boys. This information come from a Ofsted report. A high court said in 2017 that this type of action is illegal.',
 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this is unlawful.',
 "In Birmingham there is a school where girls and boys are separeted even if it isn't legal.",
 'A school in Birningham is still separating girls from boys. This information come from a Ofsted report. A high court said in 2017 that this type of action is illegal.',
 'According to a recent OFSTED report, a school in Birmingham is still segregating girls and boys, despite a Hight Court ruling in 2017 that this 

In [7]:
multi_id_ = []
multi_hate = []
multi_counter = []

for _ in range(0, len(conan_multi[0])):
    multi_id_.append(_)
    multi_hate.append(conan_multi[0][str(_)]["HATE_SPEECH"])
    multi_counter.append(conan_multi[0][str(_)]["COUNTER_NARRATIVE"])

print(len(multi_hate))

5003


In [9]:
# Clean Post

# Display Post IPython Markdown
def show_post(hate, counter, type):
    # cmv_post = cleanup(cmv_post)
    md_format = "**{}** \n \n {} \n \n **Counter** \n \n {}".format(hate, counter, type)
    md_format = "\n".join(["> " + line for line in md_format.splitlines()])

    return Markdown(md_format)

# Clean Text
def full_clean(data):
    cleaned = []
    for i in data:
        clean = i.lower()
        # RegEx clean-up
        clean = re.sub("CMV:", " ", clean)
        clean = re.sub("CMV", " ", clean)
        clean = re.sub("cmv", " ", clean)
        clean = re.sub(r'(\.)(?:[A-Z])',r'\1\n', clean)
        clean = re.sub(r"http\S+", "", clean)
        clean = re.sub(r"\n", "", clean)
        clean = re.sub(r'(?<=[a-z])\'(?=[a-z])', '', clean)
        clean = re.sub('([^a-zA-Z\s.!?])', "", clean)
        clean = re.sub('\s+', ' ', clean)
        clean = re.sub(r"www\S+", "", clean)

        cleaned.append(clean.strip())

    return cleaned

In [10]:
len(ids), len(hate), len(full_clean(hate))

(3864, 3864, 3864)

In [13]:
### CONSTRUCT DATAFRAME OBJECTS ###

hate_clean = full_clean(hate)
counter_clean = full_clean(counter)
type_clean = full_clean(type)
sub_type_clean = full_clean(sub_type)

hate_obj = {
    "id": ids,
    "hate": hate_clean,
    "counter": counter_clean,
    # "type": type_clean,
    # "sub_type": sub_type_clean,
}

multi_hate_clean = full_clean(multi_hate)
multi_counter_clean = full_clean(multi_counter)

multi_hate_obj = {
    "id": multi_id_,
    "hate": multi_hate_clean,
    "counter": multi_counter_clean,
}

hate_df = pd.DataFrame(hate_obj).astype(str)
multi_hate_df = pd.DataFrame(multi_hate_obj).astype(str)

In [14]:
# Explore Post
_ = random.randint(0, len(hate_df))
show_post(hate_df["id"][_], hate_df["hate"][_], hate_df["counter"][_])

> **ENT1ST0008HS0053CN000446P1** 
>  
>  people that live in the west cannot live with muslims. 
>  
>  **Counter** 
>  
>  maybe prejudice and xenophobia are incompatible with any western society?

In [15]:
# Explore Post
_ = random.randint(0, len(hate_df))
show_post(multi_hate_df["id"][_], multi_hate_df["hate"][_], multi_hate_df["counter"][_])

> **2021** 
>  
>  migration is the biggest contributor to poverty in our country. 
>  
>  **Counter** 
>  
>  on the contrary migration is an important part of our economy it helps to create new jobs for people and to boost our economy.

In [16]:
# Exploritory Search Keywords; Assert Clean (URLs: 'http', 'www')
keyword = "www"
hate_df[hate_df['counter'].str.contains(keyword,case=False)]

Empty DataFrame
Columns: [id, hate, counter]
Index: []

In [17]:
# Search Keywords; Assert missing values at start of sentence (known argument issue)
keyword = "harassment"
hate_df[hate_df['counter'].str.contains(keyword,case=False)]

id  \
3141    ENT1ST0016HS0094CN001048   
3142  ENT1ST0016HS0094CN001048P1   
3143  ENT1ST0016HS0094CN001048P2   

                                                   hate  \
3141  our government has brought islamic invaders to...   
3142  women and girls are harmed by islamic that wer...   
3143  our women and girls are frequently victim of r...   

                                                counter  
3141  and what do girls and women have to say about ...  
3142  and what do girls and women have to say about ...  
3143  and what do girls and women have to say about ...

In [20]:
### CONCATENATE ###
total_hate = pd.concat([hate_df, multi_hate_df])

print(len(total_hate))
total_hate

8867


id  \
0       ENT1ST0001HS0033CN000021   
1     ENT1ST0001HS0033CN000021P1   
2     ENT1ST0001HS0033CN000021P2   
3       ENT1ST0001HS0033CN000022   
4     ENT1ST0001HS0033CN000022P1   
...                          ...   
4998                        4998   
4999                        4999   
5000                        5000   
5001                        5001   
5002                        5002   

                                                   hate  \
0     according to a recent ofsted report a school i...   
1     in birmingham there is a school where girls an...   
2     a school in birningham is still separating gir...   
3     according to a recent ofsted report a school i...   
4     in birmingham there is a school where girls an...   
...                                                 ...   
4998                     woman are basically childlike.   
4999                 women should be submissive to men.   
5000  women are essentially childlike unable to unde...   
5001                 women should be submissive to men.   
5002  women should not be allowed to vote this is a ...   

                                                counter  
0     to be fair the ofsted report is more concerned...  
1     to be fair the ofsted report is more concerned...  
2     to be fair the ofsted report is more concerned...  
3     while this story is presented as an islamic is...  
4     while this story is presented as an islamic is...  
...                                                 ...  
4998  they are not childlike and are capable of grea...  
4999  there are many women who want to be submissive...  
5000  maybe you should try talking to a woman instea...  
5001  women are human beings deserving respect and e...  
5002  i dont think that a country can be male domina...  

[8867 rows x 3 columns]

In [24]:
# Output JSON List .jsonl
import json

hate_data = []
for idx, row in total_hate.iterrows():
    hate_data.append({
        "id": row["id"],
        "hate": row["hate"],
        "counter": row["counter"],
    })

with open("../data/processed/hate_cleaned.jsonl", "w", encoding='utf-8') as f:
    for d in hate_data:
        f.write(json.dumps(d))
        f.write("\n")